# Linked Lists


# Problem

Arrays have two big problems:

- Insertion/deletion in middle → **O(n)** (everything shifts!)
- Fixed size in many languages (C/C++)

Linked List = the solution!
Each node = data + pointer to next node

**Insert/delete in middle → O(1)** (just change 2–3 pointers!)

### Analogy

| Array                        | Linked List                              |
|------------------------------|------------------------------------------|
| People shoulder-to-shoulder  | People holding hands across a field       |
| Insert? Everyone shuffles!   | Just change 2 hand holds → done!         |

### Real-World Uses

- OS uses them for process queues & memory management
- Python’s `collections.deque` → built with linked list internally!
- LRU Cache → doubly linked list + hashmap

**Rule of thumb**:  
Fast random access → Array/List  
Lots of insert/delete anywhere → Linked List

# Implementation - Basic Operations


In [4]:
# Basic Operations
class LinkedList:
    def __init__(self):
        self.head = None    # starting point
    
    # 1. Insert at the beginning (O(1))
    def insert_at_front(self, data):
        new_node = Node(data)
        new_node.next = self.head
        self.head = new_node
    
    # 2. Insert at the end (O(n) unless we keep a tail pointer)
    def append(self, data):
        new_node = Node(data)
        if not self.head:
            self.head = new_node
            return
        
        current = self.head
        while current.next:        # go to last node
            current = current.next
        current.next = new_node
    
    # 3. Delete a node by value
    def delete(self, value):
        if not self.head:
            return
        
        if self.head.data == value:
            self.head = self.head.next
            return
        
        current = self.head
        while current.next:
            if current.next.data == value:
                current.next = current.next.next
                return
            current = current.next
    
    # 4. Print the list
    def print_list(self):
        current = self.head
        while current:
            print(current.data, end=" -> ")
            current = current.next
        print("None")

## Implementation - Find the middle node (tortoise-hare pointer)

In [ ]:
##Find the middle node (tortoise-hare pointer)
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def find_middle(head):
    if not head:
        return None
    
    slow = head
    fast = head
    
    # Fast moves 2 steps, slow moves 1 step
    # When fast reaches end, slow will be at the middle
    while fast and fast.next:
        slow = slow.next          # moves 1 step
        fast = fast.next.next     # moves 2 steps
    
    return slow

# Test it!
# Create list: 1 -> 2 -> 3 -> 4 -> 5
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)
head.next.next.next = Node(4)
head.next.next.next.next = Node(5)

middle = find_middle(head)
print("Middle node value:", middle.data)  # Output: 3

# Test with even number of nodes: 1 -> 2 -> 3 -> 4 -> 5 -> 6
head.next.next.next.next.next.next = Node(6)
middle = find_middle(head)
print("Middle node value (even):", middle.data)  # Output: 4 (second middle)
#Bonus: Return First Middle for Even Length?
#Some companies want the first middle (3 instead of 4). Just change the loop:
#  while fast.next and fast.next.next:  # stop one step earlier
# slow = slow.next
# fast = fast.next.next

## Implementation - Detect a cycle (Floyd’s cycle detection)

In [ ]:
##Detect a cycle (Floyd’s cycle detection)
#Imagine two runners on a circular track:

#Tortoise (slow) → walks slowly (1 step)
#Hare (fast) → runs twice as fast (2 steps)

#If the track is a straight line → hare reaches the end first → no meeting → no cycle
#If the track is a circle → hare will eventually lap the tortoise → they meet → cycle detected!
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def has_cycle(head):
    if not head or not head.next:
        return False
    
    slow = head        # Tortoise: moves 1 step
    fast = head.next   # Hare: moves 2 steps
    
    while slow != fast:
        if not fast or not fast.next:
            return False  # Fast reached end → no cycle
        
        slow = slow.next          # 1 step
        fast = fast.next.next     # 2 steps
    
    return True  # slow and fast met → cycle exists!

# Test 1: No cycle
head1 = Node(1)
head1.next = Node(2)
head1.next.next = Node(3)
head1.next.next.next = Node(4)
print(has_cycle(head1))  # False

# Test 2: With cycle
head2 = Node(1)
head2.next = Node(2)
head2.next.next = Node(3)
head2.next.next.next = Node(4)
head2.next.next.next.next = head2.next  # creates cycle: 4 → 2

print(has_cycle(head2))  # True

# Test 3: Cycle to head
head3 = Node(1)
head3.next = head3  # single node pointing to itself
print(has_cycle(head3))  # True

#Bonus: Find the Starting Point of the Cycle (Extra Credit)
def find_cycle_start(head):
    slow = fast = head
    # Step 1: Detect cycle (same as above)
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            break
    else:
        return None  # no cycle
    
    # Step 2: Reset slow to head, move both one step → they meet at start
    slow = head
    while slow != fast:
        slow = slow.next
        fast = fast.next
    
    return slow  # this is the start of the cycle

## Implementation - Merge two sorted linked lists

In [ ]:
##Merge two sorted linked lists
#List1: 1 → 3 → 5
#List2: 2 → 4 → 6
#Watch what the function does:
#Compare 1 and 2 → 1 is smaller → 1 wins
#→ Now solve the smaller problem: merge (3→5) with (2→4→6)
#Compare 3 and 2 → 2 is smaller → 2 wins
#→ Now solve: merge (3→5) with (4→6)
#Compare 3 and 4 → 3 is smaller → 3 wins
#→ Now solve: merge (5) with (4→6)
#Compare 5 and 4 → 4 is smaller → 4 wins
#→ Now solve: merge (5) with (6)
#…and so on until one list becomes empty.
def merge_sorted_lists_recursive(head1, head2):
    if not head1:          # Pile 1 is empty?
        return head2       # → just give back Pile 2
    if not head2:          # Pile 2 is empty?
        return head1       # → just give back Pile 1
    
    if head1.data <= head2.data:
        # head1 has the smaller (or equal) card → it wins!
        # Now attach the rest of head1 to the result of merging
        # the remaining parts of both lists
        head1.next = merge_sorted_lists_recursive(head1.next, head2)
        return head1
    else:
        # head2 has the smaller card → it wins!
        head2.next = merge_sorted_lists_recursive(head1, head2.next)
        return head2

## Implementation -  Find intersection of two lists

In [ ]:
##Find intersection of two lists
#Path A:    1 → 2 → 3 → 6 → 7 → 8
#Path B:        4 → 5 → 6 → 7 → 8
#                       ↑
#             They meet here!
#Alex walks: 1→2→3→6→7→8 (finishes A) → jumps to B → 4→5→6…
#Bella walks: 4→5→6→7→8 (finishes B) → jumps to A → 1→2→3→6…
#They both arrive at node 6 at the exact same time → found the intersection!
a = headA
b = headB
while a != b:
    a = headB if a is None else a.next   # teleport trick
    b = headA if b is None else b.next   # teleport trick
return a  # this is the meeting point!

## Implementation - Reverse in groups of k

In [ ]:
##Reverse in groups of k
#Given a linked list and a number k, reverse every k nodes.
#If the last group has fewer than k nodes → leave it as-is.
#Example:
#List: 1 → 2 → 3 → 4 → 5 → 6 → 7 → 8 and k = 3
#→ Reverse first 3 → reverse next 3 → last 2 stay normal
#Result: 3 → 2 → 1 → 6 → 5 → 4 → 7 → 8
#Step-by-Step Plan
#Check if there are at least k nodes left → if not, stop.
#Reverse the next k nodes (using normal reverse logic).
#Recursively do the same for the remaining list.
#Connect the reversed part to the result of the rest.
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

def reverse_k_group(head, k):
    # Step 1: Count if we have k nodes
    count = 0
    curr = head
    while curr and count < k:
        curr = curr.next
        count += 1
    
    # If we have k nodes → reverse them
    if count == k:
        # Reverse first k nodes
        curr = head
        prev = None
        for _ in range(k):
            next_node = curr.next
            curr.next = prev
            prev = curr
            curr = next_node
        
        # Now 'prev' is the new head of this reversed group
        # 'head' is now the last node → connect it to the result of rest
        head.next = reverse_k_group(curr, k)  # recursive magic!
        return prev  # new head of this group
    
    # Less than k nodes → return as-is
    return head

# Helper: Create list from array
def create_list(arr):
    if not arr:
        return None
    head = Node(arr[0])
    curr = head
    for val in arr[1:]:
        curr.next = Node(val)
        curr = curr.next
    return head

# Helper: Print list
def print_list(head):
    while head:
        print(head.data, end=" → ")
        head = head.next
    print("None")

# Test it!
head = create_list([1, 2, 3, 4, 5, 6, 7, 8])
print("Original:")
print_list(head)

reversed_head = reverse_k_group(head, 3)
print("After reversing every 3 nodes:")
print_list(reversed_head)
# Output: 3 → 2 → 1 → 6 → 5 → 4 → 7 → 8 → None

## Implementation - LRU Cache (uses doubly linked list + hashmap)

In [ ]:
##LRU Cache (uses doubly linked list + hashmap)
#Imagine a small shelf with only 3 spots for your favorite toys:
#[ Spot 3 ]  [ Spot 2 ]  [ Spot 1 ]   ← Spot 1 = most recently used
#Every time you play with a toy:
#You put it on Spot 1 (most recent)
#Everything else shifts right
#The toy that falls off the right side = least recently used → removed!
#put(1,1) → [1]
#put(2,2) → [2, 1]
#put(3,3) → [3, 2, 1]
#get(1)   → [1, 3, 2]    ← 1 moved to front
#put(4,4) → [4, 1, 3]    ← 2 was kicked out!
#Doubly Linked List - Super fast to move a toy to the front (O(1))
#Hash Map (dict)- Find any toy instantly by name (O(1))


class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}           # key → node
        
        # Dummy head and tail (makes code cleaner)
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
    
    # Remove a node from linked list
    def _remove(self, node):
        prev = node.prev
        nxt = node.next
        prev.next = nxt
        nxt.prev = prev
    
    # Add a node right after head (most recent)
    def _add(self, node):
        nxt = self.head.next
        self.head.next = node
        node.prev = self.head
        node.next = nxt
        nxt.prev = node
    
    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            self._remove(node)      # remove from current position
            self._add(node)         # move to front
            return node.value
        return -1
    
    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self._remove(node)
            self._add(node)
        else:
            new_node = Node(key, value)
            self.cache[key] = new_node
            self._add(new_node)
            
            # If over capacity → remove least recently used (tail.prev)
            if len(self.cache) > self.capacity:
                lru = self.tail.prev
                self._remove(lru)
                del self.cache[lru.key]

# Test it!
lru = LRUCache(3)

lru.put(1, 1)
lru.put(2, 2)
lru.put(3, 3)
print(lru.get(1))    # → 1 (1 becomes most recent)

lru.put(4, 4)        # evicts key=2
print(lru.get(2))    # → -1 (not found)

lru.put(3, 33)       # updates 3
print(lru.get(3))    # → 33

## Implementation - Copy list with random pointers

In [ ]:
##Copy list with random pointers
#magine you have a chain of people holding hands + each person has a “best friend” finger pointing to anyone in the chain (or no one).
#Your job: Make an exact twin copy of this whole chain, including who everyone’s best friend is.
#If you just make new people one by one and try to point to the best friend later → you get stuck!
#Because when Person A says “my best friend is Person C”, you haven’t made the twin of Person C yet!
#Step 1 – Clone and stand right next to your original
#Everyone makes their twin and the twin stands immediately behind them.
#Original:       A    B    C    D
#After step 1:   A → A' → B → B' → C → C' → D → D'
#                ↑   ↑      
#          original  twin 
#Now every original person has their twin right next to them → super easy to find!
#Step 2 – Copy the “best friend” pointers
#For every original person:
#My twin is the one standing right behind me (original.next)
#My best friend’s twin is standing right behind my best friend (original.random.next)
#So we just say:
#Pythonmy_twin.random = my_best_friend.random.next
#Done! All best-friend pointers are copied correctly.
#Step 3 – Separate the twins from the originals
#Now we carefully pull the twins out into their own chain and give the originals their normal chain back.
#Original chain again:  A → B → C → D
#Twin chain (the copy): A'→ B'→ C'→ D'
#Perfect twin copy in 3 simple steps!

class Node:
    def __init__(self, val):
        self.val = val
        self.next = None
        self.random = None

def copy_random_list(head):
    if not head:
        return None
    
    # Step 1: Insert copy nodes (A → A' → B → B' → ...)
    curr = head
    while curr:
        copy = Node(curr.val)
        copy.next = curr.next
        curr.next = copy
        curr = copy.next  # move to next original
    
    # Step 2: Set random pointers for copied nodes
    curr = head
    while curr:
        if curr.random:
            curr.next.random = curr.random.next  # original.random.next = its copy!
        curr = curr.next.next  # jump to next original
    
    # Step 3: Separate the lists
    curr = head
    copy_head = head.next
    copy_curr = copy_head
    
    while curr:
        # Restore original list
        curr.next = curr.next.next if curr.next else None
        
        # Build copy list
        copy_curr.next = copy_curr.next.next if copy_curr.next else None
        copy_curr = copy_curr.next
        
        curr = curr.next  # move to next original
    
    return copy_head

# Test it!
# Create original list: 7 → 13 → 11 → 10 → 1
# Random pointers: 7→None, 13→7, 11→1, 10→11, 1→7

def create_test_list():
    n1 = Node(7)
    n2 = Node(13)
    n3 = Node(11)
    n4 = Node(10)
    n5 = Node(1)
    
    n1.next = n2
    n2.next = n3
    n3.next = n4
    n4.next = n5
    
    n1.random = None
    n2.random = n1
    n3.random = n5
    n4.random = n3
    n5.random = n1
    
    return n1

# Print helper
def print_list_with_random(head):
    curr = head
    while curr:
        rand_val = curr.random.val if curr.random else "None"
        print(f"{curr.val} (random→{rand_val})", end=" → ")
        curr = curr.next
    print("None")

print("Original:")
print_list_with_random(create_test_list())

copy = copy_random_list(create_test_list())
print("Copy:")
print_list_with_random(copy)